In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

from JacobianODE.jacobians.jacobian_utils import load_config
from JacobianODE.jacobians.run_jacobians import train_jacobians

In [3]:
# FILL THIS IN FOR YOUR OWN USE
save_dir = "/orcd/data/ekmiller/001/eisenaj/JacobianODE/lightning"
wandb_entity = "chaotic-consciousness"

## Setup

In [4]:
# load the config
overrides = [
    "data=dysts",
    "data.flow._target_=JacobianODE.dysts_sim.flows.Lorenz",
    "data.postprocessing.obs_noise=0.01",
    "training.lightning.loop_closure_weight=0.001",
    f"training.logger.save_dir={save_dir}",
    f"wandb_entity={wandb_entity}",
]
cfg = load_config(overrides=overrides)

In [ ]:
train_jacobians(cfg)